# Notebook 2 — Copy-Move Forgery Detection

Copy-move forgery is one of the most common forms of image tampering: a region of an image is duplicated and pasted elsewhere (often to conceal or duplicate an object).

## Outline
1. Visual introduction to copy-move forgery
2. Classical approach: ORB feature matching + RANSAC
3. Modern approach: PhotoHolmes library (Splicebuster)
4. Result visualisation

## References
- **CMFDFormer** (arXiv 2311.13263): MiT transformer backbone for CMFD — https://arxiv.org/abs/2311.13263
- **PhotoHolmes** (arXiv 2412.14969, Springer 2025): unified forensics library — https://github.com/photoholmes/photoholmes
- **MVSS-Net++** (T-PAMI): multi-view multi-scale supervision — https://github.com/dong03/MVSS-Net

In [ ]:
import sys
sys.path.insert(0, '..')

from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

TEST_DIR = Path('../Copy Move Forgery/CopyMoveDetection/test_images')
test_images = list(TEST_DIR.glob('*.png')) + list(TEST_DIR.glob('*.jpg'))
print('Test images found:', [p.name for p in test_images])

## 1. Load Test Image

In [ ]:
img_path = TEST_DIR / 'cheque22.png'
if not img_path.exists():
    img_path = test_images[0]

img = Image.open(img_path).convert('RGB')
print(f'Image: {img_path.name}  |  Size: {img.size}')
plt.figure(figsize=(10, 5))
plt.imshow(img)
plt.axis('off')
plt.title('Input document image')
plt.tight_layout()
plt.show()

## 2. ORB + RANSAC Detection

In [ ]:
from src.copy_move.detector import detect_copy_move
from src.copy_move.visualizer import overlay_heatmap, annotate_regions, side_by_side

result = detect_copy_move(img_path)
print(f'Verdict:  {result["verdict"]}')
print(f'Score:    {result["score"]:.4f}')
print(f'Method:   {result["method"]}')

## 3. Visualise Results

In [ ]:
img_arr = np.array(img)
mask = result['mask']

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(img)
axes[0].set_title('Original')
axes[0].axis('off')

if mask.any():
    overlay = overlay_heatmap(img_arr, mask, alpha=0.45)
    axes[1].imshow(overlay)
    axes[1].set_title('Heatmap Overlay')
else:
    axes[1].imshow(img)
    axes[1].set_title('No copy-move detected')
axes[1].axis('off')

if mask.any():
    annotated = annotate_regions(img_arr, mask)
    axes[2].imshow(annotated)
    axes[2].set_title('Annotated Regions')
else:
    axes[2].imshow(mask, cmap='gray')
    axes[2].set_title('Detection Mask (empty)')
axes[2].axis('off')

plt.suptitle(f'Copy-Move Detection — {result["verdict"]} (score={result["score"]:.3f})', fontsize=13)
plt.tight_layout()
plt.show()

## 4. ORB Keypoint Visualisation

In [ ]:
bgr = cv2.imread(str(img_path))
gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
orb = cv2.ORB_create(nfeatures=500)
kp, _ = orb.detectAndCompute(gray, None)
kp_img = cv2.drawKeypoints(bgr, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
kp_rgb = cv2.cvtColor(kp_img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 6))
plt.imshow(kp_rgb)
plt.title(f'ORB Keypoints ({len(kp)} detected)')
plt.axis('off')
plt.tight_layout()
plt.show()